# Implementation

In [42]:
import mido
import csv
import audiolazy

In [43]:
def get_active_notes(mid):
    active_notes = {}
    tracks = mid.tracks
    num_tracks = len(tracks)
    all_notes = num_tracks * [None]
    for i in range(0, num_tracks):
        track = tracks[i]
        time = 0
        all_notes[i] = []
        for msg in track:
            msg_dict = msg.dict()
            time += msg_dict['time']
            if msg.type == 'note_on':
                vel = msg_dict['velocity']
                if vel > 0:
                    # Using a list for the active notes becuase note 71 in io.mid was definied twice at once
                    if active_notes.has_key(msg_dict['note']):
                        active_notes[msg_dict['note']].append({'time':time,'velocity':vel})
                    else:
                        active_notes[msg_dict['note']] = [{'time':time, 'velocity': vel}]
                elif vel == 0:
                    note = msg_dict['note']
                    start_msg = active_notes[note].pop()
                    new_note = {'note': note, 'start': start_msg['time'],
                                'end': time, 'velocity': start_msg['velocity']}
                    all_notes[i].append(new_note)
    return all_notes

In [44]:
def create_streams(all_notes):
    streams = []
    for notes in all_notes:
        while notes != []:
            stream = []
            vel = 0
            current_end = 0
            for note in notes:
                if note['start'] >= current_end:
                    if note['velocity'] != vel:
                        vel = note['velocity']
                    else:
                        del note['velocity']
                    stream.append(note)
                    current_end = note['end']
            streams.append(stream)
            for note in stream:
                notes.remove(note)
    return streams

In [45]:
def streams_to_cells(streams, speed):
    max_time = int(max([x['end'] for x in [item for sublist in streams for item in sublist]]))
    start_cells = 'A2:A' + str(1+len(streams))
    instructions = 'r m' + str(max_time-1)
    turtles = [['!turtle(' + start_cells + ', ' + instructions + ', ' + str(speed) + ', 1)']]
    for stream in streams:
        cells = [""] * max_time
        for note in stream:
            start = int(note['start'])
            cells[start] = audiolazy.midi2str(note['note'])
            if note.has_key('velocity'):
                cells[start] += (' ' + str(round(float(note['velocity'])/127,2)))
            for rest_duration in range(1,int(note['length'])):
                cells[start+rest_duration] = '-'
        turtles.append(cells)
    print(str(len(turtles)) + ' x ' + str(max([len(stream) for stream in turtles])))
    return turtles

In [80]:
def midi_to_excello(file_name, method=1):
    # Fetch MIDI file
    mid = mido.MidiFile(file_name) 
    tempo = [m.dict()['tempo'] for m in mid.tracks[0] if m.dict().has_key('tempo')][0]
    ticks_per_beat = mid.ticks_per_beat
    # Extract the notes from as onset, note, offset, volume from messages
    all_notes = get_active_notes(mid)
    # Split into the streams as played by individual turtles
    streams = create_streams(all_notes)
    all_notes = [item for sublist in streams for item in sublist]
    print('Number of turtles: ' + str(len(streams)))
    
    # No Compression
    if method == 0:
        print("No Compression")
        difference_stat = 1
        ratio_int = 1
        for stream in streams:
            for note in stream:
                note['length'] = note['end'] - note['start']
    #Compression
    else:
        differences = [(y['start']-x['start']) for x, y in zip(all_notes[:-1], all_notes[1:])]
        lengths = [(x['end'] - x['start']) for x in [item for sublist in streams for item in sublist]]
        # Mins
        if method == 1:
            print("Min Compression")
            difference_stat = min([x for x in differences if x > 0])
            length_stat = min([x for x in lengths if x > 0])
        # Modes 
        elif method == 2:
            print("Mode Compression")
            difference_stat = max(set(differences), key=differences.count)
            length_stat = max(set(lengths), key=lengths.count)

        print('note difference stat: ' + str(difference_stat))
        print('note length stat: ' + str(length_stat))

        mode_ratio = (float(max(difference_stat, length_stat)) / min(difference_stat, length_stat))
        print('mode ratio: ' + str(mode_ratio))
        ratio_int = int(mode_ratio)
        print('integer ratio: ' + str(ratio_int))
        ratio_correction = mode_ratio/ratio_int
        print('ratio correction: ' + str(ratio_correction))
    
        # Convert MIDI times to cell times
        rounding_base = 0.1
        for stream in streams:
            for note in stream:
                note['length'] = ((float(note['end']) - note['start'])/length_stat) 
                note['length'] = rounding_base * round(note['length']/rounding_base)
                note['start'] = round(rounding_base * round((float(note['start'])/difference_stat*ratio_int)/rounding_base))
                note['end'] = note['start'] + note['length']
            
    speed = int(round((float(60*10**6)/tempo) * ticks_per_beat * (float(ratio_int)/difference_stat)))
    print(speed)
            
    csv_name = file_name.split('.')[0] + '.csv'
    with open(csv_name, "wb") as f:
        writer = csv.writer(f)
        writer.writerows(streams_to_cells(streams, speed))
    print("Written to " + csv_name)

# Converting

0: No Compression<br>
1: Compression using Minimum difference<br>
2: Compression using Modal difference

In [81]:
file_name = 'io.mid'

In [82]:
midi_to_excello(file_name, 0)

Number of turtles: 6
No Compression
69696
7 x 32819
Written to io.csv


# Old stuff

In [58]:
m = mido.MidiFile("io.mid")

In [62]:
streams = create_streams(get_active_notes(m))

In [63]:
streams

[[{'end': 719, 'note': 83, 'start': 0, 'velocity': 49},
  {'end': 1439, 'note': 83, 'start': 720},
  {'end': 1679, 'note': 81, 'start': 1440},
  {'end': 1919, 'note': 79, 'start': 1680},
  {'end': 2159, 'note': 81, 'start': 1920},
  {'end': 2639, 'note': 83, 'start': 2160},
  {'end': 3359, 'note': 79, 'start': 2640},
  {'end': 3599, 'note': 81, 'start': 3360},
  {'end': 3839, 'note': 79, 'start': 3600},
  {'end': 4079, 'note': 78, 'start': 3840},
  {'end': 4559, 'note': 83, 'start': 4080},
  {'end': 5279, 'note': 79, 'start': 4560},
  {'end': 5519, 'note': 81, 'start': 5280},
  {'end': 5759, 'note': 79, 'start': 5520},
  {'end': 5999, 'note': 78, 'start': 5760},
  {'end': 6479, 'note': 83, 'start': 6000},
  {'end': 6959, 'note': 84, 'start': 6480},
  {'end': 7199, 'note': 84, 'start': 6960},
  {'end': 7439, 'note': 83, 'start': 7200},
  {'end': 7679, 'note': 79, 'start': 7440},
  {'end': 8399, 'note': 77, 'start': 7680},
  {'end': 9119, 'note': 81, 'start': 8400},
  {'end': 9359, 'note

#### import pygame

In [2]:
def play_music(music_file):
    """
    stream music with mixer.music module in blocking manner
    this will stream the sound from disk while playing
    """
    clock = pygame.time.Clock()
    try:
        pygame.mixer.music.load(music_file)
        print "Music file %s loaded!" % music_file
    except pygame.error:
        print "File %s not found! (%s)" % (music_file, pygame.get_error())
        return
    pygame.mixer.music.play()
    while pygame.mixer.music.get_busy():
        # check if playback has finished
        clock.tick(30)
# pick a midi music file you have ...
# (if not in working folder use full path)

freq = 44100    # audio CD quality
bitsize = -16   # unsigned 16 bit
channels = 2    # 1 is mono, 2 is stereo
buffer = 1024    # number of samples
pygame.mixer.init(freq, bitsize, channels, buffer)

In [5]:
play_music(file_name)

Music file bach_850.mid loaded!


KeyboardInterrupt: 

In [10]:
mid = mido.MidiFile(file_name)

In [17]:
list(mid.tracks[0])

[<meta message time_signature numerator=4 denominator=4 clocks_per_click=24 notated_32nd_notes_per_beat=8 time=0>,
 <meta message key_signature key='A' time=0>,
 <meta message set_tempo tempo=526316 time=0>,
 <message control_change channel=0 control=121 value=0 time=0>,
 <message program_change channel=0 program=0 time=0>,
 <message control_change channel=0 control=7 value=100 time=0>,
 <message control_change channel=0 control=10 value=64 time=0>,
 <message control_change channel=0 control=91 value=0 time=0>,
 <message control_change channel=0 control=93 value=0 time=0>,
 <meta message midi_port port=0 time=0>,
 <message note_on channel=0 note=62 velocity=80 time=0>,
 <message note_on channel=0 note=66 velocity=80 time=0>,
 <message note_on channel=0 note=62 velocity=0 time=239>,
 <message note_on channel=0 note=66 velocity=0 time=0>,
 <message note_on channel=0 note=66 velocity=80 time=241>,
 <message note_on channel=0 note=69 velocity=80 time=0>,
 <message note_on channel=0 note=66

In [11]:
mid.tracks

[<midi track '' 706 messages>, <midi track '' 600 messages>]

In [12]:
all_notes = get_active_notes(mid)

Track 0: 
Track 1: 


In [13]:
all_notes

[[{'end': 239, 'note': 62, 'start': 0, 'velocity': 80},
  {'end': 239, 'note': 66, 'start': 0, 'velocity': 80},
  {'end': 599, 'note': 66, 'start': 480, 'velocity': 80},
  {'end': 599, 'note': 69, 'start': 480, 'velocity': 80},
  {'end': 839, 'note': 69, 'start': 720, 'velocity': 80},
  {'end': 839, 'note': 73, 'start': 720, 'velocity': 80},
  {'end': 1319, 'note': 66, 'start': 1200, 'velocity': 80},
  {'end': 1319, 'note': 69, 'start': 1200, 'velocity': 80},
  {'end': 1799, 'note': 62, 'start': 1680, 'velocity': 80},
  {'end': 1799, 'note': 66, 'start': 1680, 'velocity': 80},
  {'end': 2039, 'note': 62, 'start': 1920, 'velocity': 80},
  {'end': 2279, 'note': 62, 'start': 2160, 'velocity': 80},
  {'end': 2519, 'note': 62, 'start': 2400, 'velocity': 80},
  {'end': 3839, 'note': 61, 'start': 3600, 'velocity': 80},
  {'end': 3959, 'note': 62, 'start': 3840, 'velocity': 80},
  {'end': 4199, 'note': 62, 'start': 4080, 'velocity': 80},
  {'end': 4199, 'note': 66, 'start': 4080, 'velocity': 8

Split into single note at a time tracks

In [ ]:
streams = create_streams(all_notes)

Calculate cell length

In [ ]:
all_notes = [item for sublist in streams for item in sublist]

In [ ]:
differences = [(y['start']-x['start']) for x, y in zip(all_notes[:-1], all_notes[1:])]
mode_difference = max(set(differences), key=differences.count)
min_difference = min([x for x in differences if x > 0])
mode_difference = min_difference

In [ ]:
lengths = [(x['end'] - x['start']) for x in [item for sublist in streams for item in sublist]]
mode_length = max(set(lengths), key=lengths.count)
min_length = min(lengths)
mode_length = min_length

In [ ]:
mode_difference

In [ ]:
mode_length

In [ ]:
mode = mode_length

In [ ]:
mode

In [ ]:
ratio = (float(mode_difference) / mode_length)/(mode_difference / mode_length)

In [ ]:
ratio_int = (mode_difference / mode_length)

In [ ]:
ratio

In [ ]:
ratio_int

In [ ]:
for stream in streams:
    for note in stream:
        note['length'] = ((float(note['end']) - note['start'])/mode) 

In [ ]:
streams

In [ ]:
rounding_base = 0.1
for stream in streams:
    for note in stream:
        note['length'] = rounding_base * round(note['length']/rounding_base)
        note['start'] = round(rounding_base * round((float(note['start'])/mode_difference*ratio_int)/rounding_base))
        note['end'] = note['start'] + note['length']

In [ ]:
streams

Create list for cells in that single turtle track

In [ ]:
max_time = int(max([x['end'] for x in [item for sublist in streams for item in sublist]]))
start_cells = 'A2:A' + str(1+num_turtles)
instructions = 'r m' + str(max_time-1)
speed = 1
turtles = [['!turtle(' + start_cells + ', ' + instructions + ', ' + str(speed) + ', 1)']]
for stream in streams:
    cells = [""] * max_time
    for note in stream:
        start = int(note['start'])
        cells[start] = audiolazy.midi2str(note['note'])
        if note.has_key('velocity'):
            cells[start] += (' ' + str(round(float(note['velocity'])/127,2)))
        for rest_duration in range(1,int(note['length'])):
            cells[start+rest_duration] = '-'
    turtles.append(cells)

Combine lists into csv

In [ ]:
csv_name = file_name.split('.')[0] + '.csv'
with open(csv_name, "wb") as f:
    writer = csv.writer(f)
    writer.writerows(turtles)